## fetch github repo stars

https://phung.io/blog/fetching-github-stars-and-forks-using-github-api%20copy

see ~/projects/wgong/github-PAT.md for personal access token

In [ ]:
import re
import os
import requests
import pandas as pd

from urllib.parse import urlparse

In [ ]:
MD_PATTERN = r"\[([^\]]*)\]\(([^)]+)\)"

In [3]:
def parse_github_repo_owner(url):
    # Extract owner and repo from the URL
    parts = url.split("/")
    if len(parts) < 5 or parts[2] != "github.com":
        print(f"Invalid GitHub repository URL format: {url}, expected format: https://github.com/<owner>/<repo>")
        return None

    owner = parts[3]
    repo = parts[4]
    if "." in repo:
        repo = repo.split(".")[0]
    return owner, repo

In [ ]:
# make sure to have valid GITHUB Personal Access Token
def report_github_repo_stars(valid_links):
    formatted_resp = []
    # formatted_resp.append(["link", "stars", "forks"])
    for txt,url in valid_links:
        x = get_repo_stars_and_forks(url)
        if x is None: continue
        formatted_resp.append([f"[{txt}]({url})", x.get("stars",0), x.get("forks",0)])
    return formatted_resp

In [6]:
def get_repo_stars_and_forks(url):
    """
    Fetches the number of stars and forks for a given GitHub URL,
    requires a personal access token with appropriate permissions stored in os env var: GITHUB_PAT.
    
    Args:
        url: f"https://github.com/{owner}/{repo}".
            
    Returns:
        A dictionary containing the number of stars and forks, or None on error.
    """
    owner_repo = parse_github_repo_owner(url)
    if owner_repo is None: 
        return None
    else:
        owner, repo = owner_repo
    
    # Build the API endpoint URL
    api_url = f"https://api.github.com/repos/{owner}/{repo}"
    
    # Set headers with authorization token
    token = os.getenv("API_KEY_GITHUB")
    headers = {"Authorization": f"Token {token}"}

    try:
        # Send GET request with headers
        response = requests.get(api_url, headers=headers)
        
        # Check for successful response
        response.raise_for_status()  # Raises exception for non-200 status codes
        
        # Parse JSON response
        data = response.json()
        
        return {
            "stars": data.get("stargazers_count", 0),  # Handle missing key gracefully
            "forks": data.get("forks_count", 0),
        }
    
    except requests.exceptions.RequestException as error:
        print(f"Error fetching stars and forks: {error}")
        return None

In [7]:
def parse_markdown_links(text):
    """
    Parses markdown text and extracts a list of tuples containing the display text and URL for each valid GitHub link.

    Args:
        text: The markdown text containing links.

    Returns:
        A list of tuples (display_text, url) for valid GitHub links, or None if no valid links found.
    """

    # match all markdown links (square brackets)
    matches = re.findall(MD_PATTERN, text)

    # Validate and extract links
    valid_links = []
    invalid_links = []
    for match in matches:
        display_text, url = match

        # Validate URL format and domain
        parsed_url = urlparse(url)
        if parsed_url.scheme == "https" and parsed_url.netloc == "github.com":
            valid_links.append((display_text, url))
        else:
            # print(f"Invalid GitHub link: {url}")
            invalid_links.append(url)
       
    # Return list of valid links or None
    return valid_links, invalid_links

# Example usage
markdown_text = """### Web & Desktop

- [Open WebUI](https://github.com/open-webui/open-webui)
- [Enchanted (macOS native)](https://github.com/AugustDev/enchanted)
- [Bard](https://bard.gemini.google.com)
- [Lobe Chat](https://github.com/lobehub/lobe-chat) with [Integrating Doc](https://lobehub.com/docs/self-hosting/examples/ollama)
"""

In [26]:
def get_url_text_from_markdown(md_str):
    # Find all matches in the text
    matched = re.findall(MD_PATTERN, md_str)[0]
    return matched[1], matched[0]

In [8]:
valid_links, invalid_links = parse_markdown_links(markdown_text)

print(valid_links)
if invalid_links:
    print("Invalid links...")
    print(invalid_links)

[('Open WebUI', 'https://github.com/open-webui/open-webui'), ('Enchanted (macOS native)', 'https://github.com/AugustDev/enchanted'), ('Lobe Chat', 'https://github.com/lobehub/lobe-chat')]
Invalid links...
['https://bard.gemini.google.com', 'https://lobehub.com/docs/self-hosting/examples/ollama']


In [9]:
url = "https://github.com/ollama/ollama"
# print(parse_github_repo_owner(url))
print(get_repo_stars_and_forks(url))

{'stars': 83135, 'forks': 6362}


## Ollama web-ui stars

see [README.md](https://github.com/ollama/ollama/blob/main/README.md)

In [10]:
readme_md = """
### Web & Desktop

### Web & Desktop

- [Open WebUI](https://github.com/open-webui/open-webui)
- [Enchanted (macOS native)](https://github.com/AugustDev/enchanted)
- [Hollama](https://github.com/fmaclen/hollama)
- [Lollms-Webui](https://github.com/ParisNeo/lollms-webui)
- [LibreChat](https://github.com/danny-avila/LibreChat)
- [Bionic GPT](https://github.com/bionic-gpt/bionic-gpt)
- [HTML UI](https://github.com/rtcfirefly/ollama-ui)
- [Saddle](https://github.com/jikkuatwork/saddle)
- [Chatbot UI](https://github.com/ivanfioravanti/chatbot-ollama)
- [Chatbot UI v2](https://github.com/mckaywrigley/chatbot-ui)
- [Typescript UI](https://github.com/ollama-interface/Ollama-Gui?tab=readme-ov-file)
- [Minimalistic React UI for Ollama Models](https://github.com/richawo/minimal-llm-ui)
- [Ollamac](https://github.com/kevinhermawan/Ollamac)
- [big-AGI](https://github.com/enricoros/big-AGI/blob/main/docs/config-local-ollama.md)
- [Cheshire Cat assistant framework](https://github.com/cheshire-cat-ai/core)
- [Amica](https://github.com/semperai/amica)
- [chatd](https://github.com/BruceMacD/chatd)
- [Ollama-SwiftUI](https://github.com/kghandour/Ollama-SwiftUI)
- [Dify.AI](https://github.com/langgenius/dify)
- [MindMac](https://mindmac.app)
- [NextJS Web Interface for Ollama](https://github.com/jakobhoeg/nextjs-ollama-llm-ui)
- [Msty](https://msty.app)
- [Chatbox](https://github.com/Bin-Huang/Chatbox)
- [WinForm Ollama Copilot](https://github.com/tgraupmann/WinForm_Ollama_Copilot)
- [NextChat](https://github.com/ChatGPTNextWeb/ChatGPT-Next-Web) with [Get Started Doc](https://docs.nextchat.dev/models/ollama)
- [Alpaca WebUI](https://github.com/mmo80/alpaca-webui)
- [OllamaGUI](https://github.com/enoch1118/ollamaGUI)
- [OpenAOE](https://github.com/InternLM/OpenAOE)
- [Odin Runes](https://github.com/leonid20000/OdinRunes)
- [LLM-X](https://github.com/mrdjohnson/llm-x) (Progressive Web App)
- [AnythingLLM (Docker + MacOs/Windows/Linux native app)](https://github.com/Mintplex-Labs/anything-llm)
- [Ollama Basic Chat: Uses HyperDiv Reactive UI](https://github.com/rapidarchitect/ollama_basic_chat)
- [Ollama-chats RPG](https://github.com/drazdra/ollama-chats)
- [QA-Pilot](https://github.com/reid41/QA-Pilot) (Chat with Code Repository)
- [ChatOllama](https://github.com/sugarforever/chat-ollama) (Open Source Chatbot based on Ollama with Knowledge Bases)
- [CRAG Ollama Chat](https://github.com/Nagi-ovo/CRAG-Ollama-Chat) (Simple Web Search with Corrective RAG)
- [RAGFlow](https://github.com/infiniflow/ragflow) (Open-source Retrieval-Augmented Generation engine based on deep document understanding)
- [StreamDeploy](https://github.com/StreamDeploy-DevRel/streamdeploy-llm-app-scaffold) (LLM Application Scaffold)
- [chat](https://github.com/swuecho/chat) (chat web app for teams)
- [Lobe Chat](https://github.com/lobehub/lobe-chat) with [Integrating Doc](https://lobehub.com/docs/self-hosting/examples/ollama)
- [Ollama RAG Chatbot](https://github.com/datvodinh/rag-chatbot.git) (Local Chat with multiple PDFs using Ollama and RAG)
- [BrainSoup](https://www.nurgo-software.com/products/brainsoup) (Flexible native client with RAG & multi-agent automation)
- [macai](https://github.com/Renset/macai) (macOS client for Ollama, ChatGPT, and other compatible API back-ends)
- [Olpaka](https://github.com/Otacon/olpaka) (User-friendly Flutter Web App for Ollama)
- [OllamaSpring](https://github.com/CrazyNeil/OllamaSpring) (Ollama Client for macOS)
- [LLocal.in](https://github.com/kartikm7/llocal) (Easy to use Electron Desktop Client for Ollama)
- [Ollama with Google Mesop](https://github.com/rapidarchitect/ollama_mesop/) (Mesop Chat Client implementation with Ollama)
- [Kerlig AI](https://www.kerlig.com/) (AI writing assistant for macOS)
- [AI Studio](https://github.com/MindWorkAI/AI-Studio)
- [Sidellama](https://github.com/gyopak/sidellama) (browser-based LLM client)
- [LLMStack](https://github.com/trypromptly/LLMStack) (No-code multi-agent framework to build LLM agents and workflows)
- [BoltAI for Mac](https://boltai.com) (AI Chat Client for Mac)
- [Harbor](https://github.com/av/harbor) (Containerized LLM Toolkit with Ollama as default backend)


"""

In [11]:
valid_links, invalid_links = parse_markdown_links(readme_md)

for i in valid_links:
    print(i)

('Open WebUI', 'https://github.com/open-webui/open-webui')
('Enchanted (macOS native)', 'https://github.com/AugustDev/enchanted')
('Hollama', 'https://github.com/fmaclen/hollama')
('Lollms-Webui', 'https://github.com/ParisNeo/lollms-webui')
('LibreChat', 'https://github.com/danny-avila/LibreChat')
('Bionic GPT', 'https://github.com/bionic-gpt/bionic-gpt')
('HTML UI', 'https://github.com/rtcfirefly/ollama-ui')
('Saddle', 'https://github.com/jikkuatwork/saddle')
('Chatbot UI', 'https://github.com/ivanfioravanti/chatbot-ollama')
('Chatbot UI v2', 'https://github.com/mckaywrigley/chatbot-ui')
('Typescript UI', 'https://github.com/ollama-interface/Ollama-Gui?tab=readme-ov-file')
('Minimalistic React UI for Ollama Models', 'https://github.com/richawo/minimal-llm-ui')
('Ollamac', 'https://github.com/kevinhermawan/Ollamac')
('big-AGI', 'https://github.com/enricoros/big-AGI/blob/main/docs/config-local-ollama.md')
('Cheshire Cat assistant framework', 'https://github.com/cheshire-cat-ai/core')
('

In [36]:
# the following links may not have public github
for link in invalid_links:
    print(link)

https://mindmac.app
https://msty.app
https://docs.nextchat.dev/models/ollama
https://lobehub.com/docs/self-hosting/examples/ollama
https://www.nurgo-software.com/products/brainsoup
https://www.kerlig.com/
https://boltai.com


In [13]:
# data = report_github_repo_stars(valid_links[:2])
data = report_github_repo_stars(valid_links)

In [14]:
data

[['[Open WebUI](https://github.com/open-webui/open-webui)', 34154, 3816],
 ['[Enchanted (macOS native)](https://github.com/AugustDev/enchanted)',
  2762,
  174],
 ['[Hollama](https://github.com/fmaclen/hollama)', 116, 17],
 ['[Lollms-Webui](https://github.com/ParisNeo/lollms-webui)', 4139, 521],
 ['[LibreChat](https://github.com/danny-avila/LibreChat)', 15936, 2661],
 ['[Bionic GPT](https://github.com/bionic-gpt/bionic-gpt)', 1805, 168],
 ['[HTML UI](https://github.com/rtcfirefly/ollama-ui)', 677, 113],
 ['[Saddle](https://github.com/jikkuatwork/saddle)', 57, 6],
 ['[Chatbot UI](https://github.com/ivanfioravanti/chatbot-ollama)', 1331, 221],
 ['[Chatbot UI v2](https://github.com/mckaywrigley/chatbot-ui)', 27704, 7695],
 ['[Typescript UI](https://github.com/ollama-interface/Ollama-Gui?tab=readme-ov-file)',
  259,
  46],
 ['[Minimalistic React UI for Ollama Models](https://github.com/richawo/minimal-llm-ui)',
  228,
  41],
 ['[Ollamac](https://github.com/kevinhermawan/Ollamac)', 1074, 57

In [20]:
df = pd.DataFrame(data, columns=["link", "stars", "forks"]).sort_values(by='stars', ascending=False)

In [21]:
df

,link,stars,forks
22,[NextChat](https://github.com/ChatGPTNextWeb/C...,73589,58342
18,[Dify.AI](https://github.com/langgenius/dify),40113,5491
37,[Lobe Chat](https://github.com/lobehub/lobe-chat),36291,8612
0,[Open WebUI](https://github.com/open-webui/ope...,34154,3816
9,[Chatbot UI v2](https://github.com/mckaywrigle...,27704,7695
20,[Chatbox](https://github.com/Bin-Huang/Chatbox),20142,2048
28,[AnythingLLM (Docker + MacOs/Windows/Linux nat...,18225,1978
4,[LibreChat](https://github.com/danny-avila/Lib...,15936,2661
34,[RAGFlow](https://github.com/infiniflow/ragflow),13275,1308
13,[big-AGI](https://github.com/enricoros/big-AGI...,5008,1124


In [ ]:
df["url"] = df["link"].map(lambda x: get_url_text_from_markdown(x)[0])
df["name"] = df["link"].map(lambda x: get_url_text_from_markdown(x)[1])

In [32]:
df.head()

,link,stars,forks,url,name
22,[NextChat](https://github.com/ChatGPTNextWeb/C...,73589,58342,https://github.com/ChatGPTNextWeb/ChatGPT-Next...,NextChat
18,[Dify.AI](https://github.com/langgenius/dify),40113,5491,https://github.com/langgenius/dify,Dify.AI
37,[Lobe Chat](https://github.com/lobehub/lobe-chat),36291,8612,https://github.com/lobehub/lobe-chat,Lobe Chat
0,[Open WebUI](https://github.com/open-webui/ope...,34154,3816,https://github.com/open-webui/open-webui,Open WebUI
9,[Chatbot UI v2](https://github.com/mckaywrigle...,27704,7695,https://github.com/mckaywrigley/chatbot-ui,Chatbot UI v2


In [33]:
df.columns

Index(['link', 'stars', 'forks', 'url', 'name'], dtype='object')

In [34]:
df_out = df[['name', 'stars', 'forks', 'url']]

In [37]:
df_out.to_csv("ollama-ui-stars-forks-v2.csv", index=False)

In [38]:
df_out.head()

,name,stars,forks,url
22,NextChat,73589,58342,https://github.com/ChatGPTNextWeb/ChatGPT-Next...
18,Dify.AI,40113,5491,https://github.com/langgenius/dify
37,Lobe Chat,36291,8612,https://github.com/lobehub/lobe-chat
0,Open WebUI,34154,3816,https://github.com/open-webui/open-webui
9,Chatbot UI v2,27704,7695,https://github.com/mckaywrigley/chatbot-ui


In [17]:
!pwd

/home/gongai/projects/wgong/py4kids/lesson-18-ai/Ollama
